In [1]:
from zero import ZeroClient
import client_data
import client_ml
import client_plot
import client_stat
from prefect import task, Flow, Parameter
from prefect.tasks.prefect import create_flow_run, wait_for_flow_run
import time

In [2]:
client1 = ZeroClient("localhost", 5559)
client2 = ZeroClient("localhost", 5558)
client3 = ZeroClient("localhost", 5557)
clients = [client1, client2, client3]

In [3]:
dfs = []
for i in range(len(clients)):
    data_id =  "cardio_70000_" + str(i) + ".txt"
    df = client_data.client_csv_to_df(clients[i], data_id, sep = ';')
    dfs.append(df)

In [4]:
dfs

[{'id': 'ad85a3c1aed9465282191d361bdb0c6d',
  'type': "<class 'pandas.core.frame.DataFrame'>",
  '_SecretClientObject': True},
 {'id': '4ac76e9a6aec44f5aeb99fae6448f311',
  'type': "<class 'pandas.core.frame.DataFrame'>",
  '_SecretClientObject': True},
 {'id': 'b025410d877744819774e958b0d05e78',
  'type': "<class 'pandas.core.frame.DataFrame'>",
  '_SecretClientObject': True}]

In [5]:
df_height_cols = []
for i in range(len(clients)):
    height_col = client_data.client_df_get_col(clients[i], dfs[i], "height")
    df_height_cols.append(height_col)

In [6]:
df_height_cols

[{'id': '259e906de4e541b0bf757be03468669b',
  'type': "<class 'pandas.core.series.Series'>",
  '_SecretClientObject': True},
 {'id': 'd29b8e8ee14743d7a193ede507552b60',
  'type': "<class 'pandas.core.series.Series'>",
  '_SecretClientObject': True},
 {'id': '64dbcb8d0e3346ada0baa2e2df005d84',
  'type': "<class 'pandas.core.series.Series'>",
  '_SecretClientObject': True}]

In [7]:
fed_mean = client_stat.fed_mean(clients, df_height_cols)
fed_mean

164.35922857142856

In [8]:
fed_min_max = client_stat.fed_min_max(clients, df_height_cols)
fed_min_max

(54.768564816137605, 250.11703191519757)

In [9]:
fed_skewness = client_stat.fed_skewness(clients, df_height_cols)
fed_skewness

-0.6421736909174576

In [10]:
fed_kurtosis = client_stat.fed_kurtosis(clients, df_height_cols)
fed_kurtosis

7.942999474186861

In [11]:
fed_variance = client_stat.fed_variance(clients, df_height_cols)
fed_variance

67.40617492168882

In [12]:
df_ap_hi_cols = []
for i in range(len(clients)):
    ap_hi = client_data.client_df_get_col(clients[i], dfs[i], "ap_hi")
    df_ap_hi_cols.append(ap_hi)

In [13]:
fed_pearson = client_stat.fed_pearson(clients, df_height_cols, df_ap_hi_cols)
fed_pearson

(0.0054877641584091, 69998)

In [15]:
import numpy as np
import math
mu = 164.35922857142856
sigma = math.sqrt(67.40617492168882)
norm_sample_1 = np.random.normal(mu, sigma, 34999)
norm_sample_1 = np.round(norm_sample_1).astype(int).tolist()
norm_sample_2 = np.random.normal(mu, sigma, 12254)
norm_sample_2 = np.round(norm_sample_2).astype(int).tolist()
norm_sample_3 = np.random.normal(mu, sigma, 22747)
norm_sample_3 = np.round(norm_sample_3).astype(int).tolist()
height_norm_sample = [norm_sample_1, norm_sample_2, norm_sample_3]

In [16]:
chi2 = client_stat.fed_chi_square(clients, df_height_cols, height_norm_sample)
chi2

(6241.296505399152, 0.9999999999999956)

In [17]:
levene = client_stat.fed_levene(clients, df_height_cols, height_norm_sample, [mu, mu, mu], [mu, mu, mu])
levene

(83.0143763223708, 139998)

In [18]:
paired_t = client_stat.fed_paired_t(clients, df_height_cols, height_norm_sample)
paired_t

(0.9320423129641423, 69999)

In [19]:
unpaired_t = client_stat.fed_unpaired_t(clients, df_height_cols, height_norm_sample)
unpaired_t

(0.9333230645609402, 139996.3859239811)

In [ ]:
#fed_kolmogorov_smirnovdef

In [ ]:
#fed_mann_whitney

In [ ]:
#fed_wilcoxon_signed_rank